In [1]:

import os
from collections import defaultdict

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [2]:
import numpy as np
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load tokenizer and model
model_name = "DeepPavlov/rubert-base-cased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)  # 6 labels for your task
#load from file
tokenizer = AutoTokenizer.from_pretrained("rubert-classifier")
model = AutoModelForSequenceClassification.from_pretrained("rubert-classifier")

# Заморозка всех слоев, кроме последнего
for param in model.parameters():
    param.requires_grad = False  # Замораживаем все параметры

# Размораживаем параметры последнего слоя (classifier)
for param in model.classifier.parameters():
    param.requires_grad = True  # Размораживаем параметры последнего слоя

# Проверка
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")


2024-12-08 08:46:14.255941: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 08:46:14.269999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-08 08:46:14.285088: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-08 08:46:14.290293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 08:46:14.302951: I tensorflow/core/platform/cpu_feature_guar

bert.embeddings.word_embeddings.weight: False
bert.embeddings.position_embeddings.weight: False
bert.embeddings.token_type_embeddings.weight: False
bert.embeddings.LayerNorm.weight: False
bert.embeddings.LayerNorm.bias: False
bert.encoder.layer.0.attention.self.query.weight: False
bert.encoder.layer.0.attention.self.query.bias: False
bert.encoder.layer.0.attention.self.key.weight: False
bert.encoder.layer.0.attention.self.key.bias: False
bert.encoder.layer.0.attention.self.value.weight: False
bert.encoder.layer.0.attention.self.value.bias: False
bert.encoder.layer.0.attention.output.dense.weight: False
bert.encoder.layer.0.attention.output.dense.bias: False
bert.encoder.layer.0.attention.output.LayerNorm.weight: False
bert.encoder.layer.0.attention.output.LayerNorm.bias: False
bert.encoder.layer.0.intermediate.dense.weight: False
bert.encoder.layer.0.intermediate.dense.bias: False
bert.encoder.layer.0.output.dense.weight: False
bert.encoder.layer.0.output.dense.bias: False
bert.encoder

In [3]:
for param in model.parameters():
    param.requires_grad = True  # Замораживаем все параметры

# Размораживаем параметры последнего слоя (classifier)
for param in model.classifier.parameters():
    param.requires_grad = True  # Размораживаем параметры последнего слоя


In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [5]:
#read train csv
df = pd.read_csv('datasets/train.csv')
X = df['TEXT']
Y = df['TOPIC']
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [6]:
print(X.shape)
print(Y.shape)
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(1000,)
(1000,)
(800,)
(800,)
(200,)
(200,)


In [7]:

labels = Y.unique()
print(labels)
#get index of labels in Y
Y = Y.apply(lambda x: list(labels).index(x))
Y_train = Y_train.apply(lambda x: list(labels).index(x))
Y_val = Y_val.apply(lambda x: list(labels).index(x))
print(Y_val)

['политика' 'туризм и путешествия' 'здоровье' 'наука и техника'
 'развлечения' 'спорт']
521    2
737    2
740    0
660    0
411    4
      ..
408    2
332    0
208    0
613    3
78     4
Name: TOPIC, Length: 200, dtype: int64


In [8]:
class_weights = (1000 / Y.value_counts())
print(class_weights)

TOPIC
0      2.061856
2      3.717472
3      7.936508
4     13.333333
1     28.571429
5    100.000000
Name: count, dtype: float64


In [9]:

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }


# Example dataset
# Corresponding labels
max_length = 512
# dataset = TextClassificationDataset(X, Y, tokenizer, max_length)
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
dataset_train = TextClassificationDataset(X_train.tolist(), Y_train.tolist(), tokenizer, max_length)
dataset_test = TextClassificationDataset(X_val.tolist(), Y_val.tolist(), tokenizer, max_length)
dataset = TextClassificationDataset(X.tolist(), Y, tokenizer, max_length)
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [10]:
optimizer = AdamW(model.parameters(), lr=1e-5)

# Learning rate scheduler
num_epochs = 20
num_training_steps = len(dataloader_train) * num_epochs

# Linear learning rate scheduler with warm-up
num_warmup_steps = int(0.1 * num_training_steps)  # 10% of training steps for warm-up
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [11]:
def f1_macro_loss(logits, labels, epsilon=1e-8):
    """Approximate F1 Macro Loss."""
    probs = torch.softmax(logits, dim=1)
    one_hot_labels = torch.nn.functional.one_hot(labels, num_classes=logits.size(1)).float()

    # Compute True Positives, False Positives, False Negatives
    tp = (probs * one_hot_labels).sum(dim=0)
    fp = (probs * (1 - one_hot_labels)).sum(dim=0)
    fn = ((1 - probs) * one_hot_labels).sum(dim=0)

    # Precision and Recall
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    # F1 Score
    f1_per_class = 2 * (precision * recall) / (precision + recall + epsilon)
    f1_macro = f1_per_class.mean()

    # Loss (maximize F1 -> minimize -F1)
    return 1 - f1_macro

In [12]:
def calculate_accuracy(dataloader, name):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:  # Replace with validation dataloader
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"{name} Accuracy: {accuracy * 100:.2f}%")


In [13]:
from sklearn.metrics import f1_score

def calculate_f1_macro(dataloader, model):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    return f1_score(all_labels, all_preds, average="macro")


In [16]:
from sklearn.utils import compute_class_weight

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
class_weights = np.sqrt(compute_class_weight('balanced', classes=np.array([0, 1, 2, 3, 4, 5]), y=Y.tolist()))
class_weights = torch.tensor(class_weights, dtype=torch.float)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

for epoch in range(num_epochs):
    model.train()
    loop = tqdm(dataloader_train, leave=True)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())
    calculate_accuracy(dataloader_train, "Train")
    calculate_accuracy(dataloader_test, "Test")
    print(calculate_f1_macro(dataloader_train, model))
    print(calculate_f1_macro(dataloader_test, model))


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:30<00:00,  1.20s/it, loss=0.00339]


Train Accuracy: 99.75%
Test Accuracy: 100.00%
0.9953153989295224
1.0


Epoch 1:   0%|                                                                                                                                                   | 0/25 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [46]:
model.save_pretrained("rubert-classifier")
tokenizer.save_pretrained("rubert-classifier")


('rubert-classifier/tokenizer_config.json',
 'rubert-classifier/special_tokens_map.json',
 'rubert-classifier/vocab.txt',
 'rubert-classifier/added_tokens.json',
 'rubert-classifier/tokenizer.json')

In [21]:
class TextClassifier:
    def __init__(self, model_path="rubert-classifier"):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.labels = ['политика','туризм и путешествия','здоровье','наука и техника',
                       'развлечения','спорт']
    
    def predict(self, text):
        inputs = self.tokenizer(
            text,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        inputs = {key: value.to(device) for key, value in inputs.items()}
        outputs = self.model(**inputs)
        predicted_label = torch.argmax(outputs.logits, dim=-1).item()
        return self.labels[predicted_label]
    
    def predict_batch(self, texts):
        result = []
        for text in texts:
            result.append(self.predict(text))
        return result

In [22]:
classifier = TextClassifier("rubert-classifier")

In [23]:
classifier.predict("Завтра еду на олимпиаду, хочу пробежать быстрее всех")

'туризм и путешествия'